In [1]:
# 通过意见表的意见，遍历文档找到仅出现一次的case，生成一个mark csv，即标志为科研通过code自动替换的。方便后面进行人工审核。
import csv
import docx
import xlrd
from tqdm import tqdm
import utils

In [2]:
# config
xlsx_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第三次会议/ALL2.xlsx'

# in_path = '/home/gray/desktop/STANDARD/test2.docx' # example
in_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第三次会议/11.11.docx'

out_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第三次会议/out.docx'

doc = docx.Document(in_path)

In [3]:
# init replace dicts
replace = {}
replace_table = {}

workbook = xlrd.open_workbook(filename=xlsx_path)
worksheet = workbook.sheet_by_name(workbook.sheet_names()[0])

In [4]:
for i in range(3, worksheet.nrows):
    row = worksheet.row_values(i)
#     sec_num = utils.get_sec_num(str(row[1]))
    sec_num = utils.get_sec_num_new(str(row[1]),from_head=True)
    
    pre_result = row[7]  # 0 -> deny, 1-> yes
    discuss = row[8]  # discuss flag
    global_index = row[10]
    
    if discuss != 1 and pre_result != 0:  # 即 没有需要讨论 且 没有被拒绝
        if sec_num is not None:  # 
            # tmp solution: self-increment for a new added-in section
            tmp_s = [int(x) for x in sec_num.split('.')]
            tmp_s[0] += 1

            sec_num = ".".join([str(x) for x in tmp_s])
            
            if sec_num not in replace.keys():
                replace[sec_num] = []
            replace[sec_num].append((row[2],row[3],row[10]))  # sec_num : (before, after, index)
        else:  # for table
            replace_table[row[2]] = (row[3],row[10])  # before : (after, index)

In [5]:
# for k,v in replace.items():
#     for vv in v:
#         if vv[0] == '使用外部存储或是本地与外部混合的存储的系统称为存储虚拟化。':
#             print(k,vv)
# #             kk = k
#             break

In [6]:
# stat info
len_replace = 0
for v in replace.values():
    len_replace += len(v)
print("len of replace in text",len_replace)
print("len of replace in text",len(replace_table))

len of replace in text 392
len of replace in text 72


In [7]:
# replace里存的是xlsx里的替换规则
# ！！！！！！不check sec了，做全文的！！！！！！
replace_global = []
for v in replace.values():
    replace_global += v

# print(len(replace_global))
# print(replace_global[:10])
found = {}
for para in tqdm(list(doc.paragraphs)):
        for a,b,index in replace_global:
            if a != '' and a in para.text:
                if az not in found.keys():
                    found[a] = [0,b,index]
                found[a][0] += 1

100%|████████████████████████████████████████████████████████████████████████| 12046/12046 [00:50<00:00, 238.08it/s]


In [16]:
key = "块服务中定义的空StoragePool，并添加了“空间限制”，“空间限制确定”和ThinProvisionMetaDataSpace属性。"

key in [i[0] for i in replace_global]

# for para in tqdm(list(doc.paragraphs)):
#     if key in para.text:
#         print(para.text)
#         break

True

In [79]:
# # !!!!!这个是做sec内部只出现了一次的。
# # 搜索段落 每个只关联自己section内部（即section内部只出现一次）
# found = {}  # 格式 "before : [show_counter, index]"
# # search in paragraphs
# # now_sec = ''
# for para in tqdm(list(doc.paragraphs)):
#     # update sec_num
    
# #     sec_num = utils.get_sec_num(para.text)
# #     if sec_num is not None:
# #         now_sec = sec_num
# #     if now_sec == '':
# #         continue
# #     if now_sec not in replace.keys():
# #         continue

#     for sec in replace.keys():
#         for a,b,index in replace[sec]:
#             if a != '' and a in para.text:
#                 if a not in found.keys():
#                     found[a] = [0,b,index]
#                 found[a][0] += 1

100%|████████████████████████████████████████████████████████████████████████| 12046/12046 [00:36<00:00, 332.45it/s]


In [12]:
# 搜索表格
found_table = {}  # 格式 "before : [counter, afer, index]"
for table_index in tqdm(range(len(list(doc.tables)))):
# for table_index in tqdm(range(len(list(doc.tables)))[10:]):
    table = doc.tables[table_index]
    for row in table.rows:
        for cell in row.cells:
            for para in cell.paragraphs:
                for a,b in replace_table.items():
                    if a!='' and a in para.text:
                        if a not in found_table.keys():
                            found_table[a] = [0]+list(b)
                        found_table[a][0] += 1
# print(found_table)

100%|█████████████████████████████████████████████████████████████████████████████| 723/723 [00:18<00:00, 38.28it/s]


In [13]:
# search有匹配的数量
print(len(found))
print(len(found_table))

216
38


In [18]:
found[key]

[1,
 '块服务中定义的空StoragePool，添加了SpaceLimit、SpaceLimitDetermination和ThinProvisionMetaDataSpace属性 ',
 572.0]

In [24]:
# 找到只出现一次的
indexes = []
for k,v in found.items():
    if v[0] ==  1:
        indexes.append(v[2])

for k,v in found_table.items():
    if v[0] <= 1:
        indexes.append(v[2])
        
print(len(indexes))
print(sorted(indexes)[:10])

219
[11.0, 14.0, 15.0, 16.0, 17.0, 31.0, 42.0, 43.0, 45.0, 46.0]


In [25]:
sorted(indexes)

[11.0,
 14.0,
 15.0,
 16.0,
 17.0,
 31.0,
 42.0,
 43.0,
 45.0,
 46.0,
 47.0,
 48.0,
 51.0,
 52.0,
 53.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 71.0,
 93.0,
 94.0,
 98.0,
 108.0,
 112.0,
 121.0,
 131.0,
 141.0,
 146.0,
 153.0,
 158.0,
 170.0,
 171.0,
 172.0,
 173.0,
 176.0,
 177.0,
 178.0,
 179.0,
 180.0,
 181.0,
 182.0,
 185.0,
 186.0,
 187.0,
 188.0,
 190.0,
 193.0,
 196.0,
 199.0,
 203.0,
 206.0,
 207.0,
 208.0,
 209.0,
 211.0,
 216.0,
 218.0,
 219.0,
 221.0,
 222.0,
 229.0,
 230.0,
 231.0,
 237.0,
 243.0,
 244.0,
 248.0,
 251.0,
 253.0,
 254.0,
 255.0,
 257.0,
 261.0,
 263.0,
 264.0,
 265.0,
 266.0,
 267.0,
 268.0,
 270.0,
 273.0,
 274.0,
 276.0,
 279.0,
 280.0,
 281.0,
 282.0,
 283.0,
 284.0,
 285.0,
 288.0,
 291.0,
 295.0,
 296.0,
 297.0,
 298.0,
 299.0,
 302.0,
 305.0,
 310.0,
 311.0,
 323.0,
 325.0,
 327.0,
 330.0,
 331.0,
 334.0,
 335.0,
 336.0,
 341.0,
 344.0,
 345.0,
 346.0,
 353.0,
 356.0,
 357.0,
 358.0,
 365.0,
 366.0,
 367.0,
 371.0,
 373.0,
 374.0,
 375.0,
 376.0,
 

In [26]:
indexes_sorted = sorted([int(i) for i in indexes])
data = []
for i in range(1,int(indexes_sorted[-1])+1):
    if i in indexes_sorted:
        data.append([1])
    else:
        data.append([])

In [27]:
# please merge this detect results to xlsx manually
with open('/home/gray/desktop/detect.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [18]:
# -------------------------------below is for testing------------------------------------------------

In [78]:
now_sec = ''
for para in tqdm(list(doc.paragraphs)):
    sec_num = utils.get_sec_num(para.text)
    if sec_num is not None:
        now_sec = sec_num
    if now_sec == '':
        continue
    if "插入到底板或是母板的系统应该使用主机硬件RAID。" in para.text:
        print(para.text)
#         break
    

100%|██████████████████████████████████████████████████████████████████████| 12046/12046 [00:00<00:00, 44714.16it/s]

阵列配置文件描述了独立冗余磁盘阵列(RAID)系统。这个配置文件描述的RAID系统是独立的，它使用本地磁盘来存储数据。使用外部存储或是本地与外部混合的存储的系统称为存储虚拟化。插入到底板或是母板的系统应该使用主机硬件RAID。该文件见第7部分主机元素。另外，为支持广域网存储管理，阵列配置文件定义了RAID系统配置服务代理（service agent，SA）,和向S-P2PN中的管理节点（directory agent, DA）注册和注销的操作。


In [22]:
now_sec

'4.1.2'

In [23]:
utils.get_sec_num("4.1.2 概述")

'4.1.2'